# HOW TO CREATE A PROJECT from scratch

+ this notebook is a template to generate Backend-required files to view a project with the DataDiVR (preview or VR)
+ STEP 1 and the "create a graph" section contains a template graph writing a required format (json) to then use the generate-project functions of the DataDiVR backend

+ STEP 2 to actually generate BACKEND project files.

In [1]:
import networkx as nx
import json 
import os
import pandas as pd

# these are the two functions one needs to create a JSON file to upload and create the project in the backend 
import nx2json as nx2j 
import uploaderGraph as uG

In [4]:
G = nx.read_edgelist("teapot40_links.csv", delimiter=',', nodetype=int)
print("Number of nodes: ", len(G.nodes()))
print("Number of Links: ", len(G.edges()))

# ===============================================
# GRAPH NAME AND DESCRIPTION - a string each
# ===============================================

G.graph['projectname'] = "Teapot_test03"
G.graph['info'] = "A toy graph for testing purposes. Number of nodes: "+str(len(G.nodes()))+", Links: "+ str(len(G.edges()))+"."

Number of nodes:  51361
Number of Links:  102560


### create node anntotations

In [5]:
import random 
def generate_random_words(num):
    words = ["alpha", "beta", "gamma", "delta", "epsilon", "zeta", "theta", "lambda", "mu", "nu"]
    return random.sample(words, num)

# Create a list to hold annotations in JSON format
l_annotations_json = []

# Process each node in the graph
for g in G.nodes():
    # Generate random annotations
    annotations = {
        "annot1": generate_random_words(random.randint(3, 4)),
        "annot2": generate_random_words(random.randint(1, 2)),
        "annot3": generate_random_words(random.randint(2, 3))
    }
    
    l_annotations_json.append(annotations)

# Create a dictionary mapping nodes to their annotations
d_annotations = dict(zip(G.nodes(), l_annotations_json))

# Set the node attributes in the graph
nx.set_node_attributes(G, d_annotations, name="annotation")


In [6]:
nodepos = pd.read_csv("teapot40_nodes.csv", delimiter=',',header =None)
pos = dict(zip(G.nodes(), zip(nodepos[0], nodepos[1], nodepos[2])))

### create node positions and set as "pos" Graph attribute 
here are 3 different layouts, which all are stored in unique nx.Graph-objects (G_rgba, G_hex, ....)

In [7]:
# First layout (i.e. Graph 1)
G_rgba = G.copy()
G_rgba.graph["layoutname"] ='layout1-teapot'
nx.set_node_attributes(G_rgba, pos, name="pos")

# Second layout (i.e. Graph 2)
G_hex = G.copy()
G_hex.graph["layoutname"] = 'layout2-teapot'
nx.set_node_attributes(G_hex, pos, name="pos")


# Fourth layout (i.e. Graph 4) - with clusters
G_clusters = G.copy()
G_clusters.graph["layoutname"] = 'layout3-teapotclusters'
clustername_1 = 'cluster group 1'
clustername_2 = 'cluster group 2'
clustername_3 = 'cluster group 3'

# nodes into groups
for g in G_clusters.nodes():
    if g < len(G_clusters.nodes()) / 3:
        G_clusters.nodes[g]['cluster'] = clustername_1
    elif g < 2 * len(G_clusters.nodes()) / 3:
        G_clusters.nodes[g]['cluster'] = clustername_2
    else:
        G_clusters.nodes[g]['cluster'] = clustername_3

nx.set_node_attributes(G_clusters, pos, name="pos")

#### node and link colors 

In [8]:
# 3 Formats of colors values are supported: hex, rgba, hex8
# FIRST GRAPH - rgba color values
d_nodecolors_rgba = {node: (0, 0, 255, 120) for node in G_rgba.nodes()}
nx.set_node_attributes(G_rgba, d_nodecolors_rgba, name="nodecolor")
nx.set_edge_attributes(G_rgba, {edge: (0, 255, 0, 100) for edge in G_rgba.edges()}, name="linkcolor")

# SECOND GRAPH - hex color values
d_nodecolors_hex = {node: '#33fcff' for node in G_hex.nodes()}
nx.set_node_attributes(G_hex, d_nodecolors_hex, name="nodecolor")

# link colors
d_linkcolors_new = {edge: '#0000ff' if int(edge[0]) % 2 == 0 else (0, 0, 0, 0) for edge in G_hex.edges()}
nx.set_edge_attributes(G_hex, d_linkcolors_new, name="linkcolor")

# THIRD GRAPH - clusters assigned
# node colors
d_nodecolors_clusters = {}
for n in G_clusters.nodes():
    cluster = G_clusters.nodes[n]['cluster']
    if cluster == clustername_1:
        d_nodecolors_clusters[n] = '#0000ff'
    elif cluster == clustername_2:
        d_nodecolors_clusters[n] = '#00ff00'
    elif cluster == clustername_3:
        d_nodecolors_clusters[n] = '#ff0000'

nx.set_node_attributes(G_clusters, d_nodecolors_clusters, name="nodecolor")

# link colors
d_linkcolors_clusters = {}
for edge in G_clusters.edges():
    cluster1 = G_clusters.nodes[edge[0]]['cluster']
    cluster2 = G_clusters.nodes[edge[1]]['cluster']
    if cluster1 == cluster2:
        if cluster1 == clustername_1:
            d_linkcolors_clusters[edge] = '#0000ff'
        elif cluster1 == clustername_2:
            d_linkcolors_clusters[edge] = '#00ff00'
        elif cluster1 == clustername_3:
            d_linkcolors_clusters[edge] = '#ff0000'
    else:
        d_linkcolors_clusters[edge] = (0, 0, 0, 0)

nx.set_edge_attributes(G_clusters, d_linkcolors_clusters, name="linkcolor")


In [9]:
# dropping random links in a graph 
num_links_to_drop = int(len(list(G_rgba.edges()))*0.5)
links_to_drop = random.sample(list(G_rgba.edges()), num_links_to_drop)
G_rgba.remove_edges_from(links_to_drop)
print("Number of links after dropping: ", len(G_rgba.edges()))

num_links_to_drop = int(len(list(G_hex.edges()))*0.7) 
links_to_drop = random.sample(list(G_hex.edges()), num_links_to_drop)
G_hex.remove_edges_from(links_to_drop)
print("Number of links after dropping: ", len(G_hex.edges()))

num_links_to_drop = int(len(list(G_clusters.edges()))*0.9) 
links_to_drop = random.sample(list(G_clusters.edges()), num_links_to_drop)
G_clusters.remove_edges_from(links_to_drop)
print("Number of links after dropping: ", len(G_clusters.edges()))

Number of links after dropping:  51280
Number of links after dropping:  30768
Number of links after dropping:  10256


### merge Graph files into one and store as a json file

In [10]:
Graphs = [G_rgba, G_hex, G_clusters]
Graphs 

# CREATE A PROJECT WITH Nx.Graph object(s)

In [11]:
nx2j.create_project(Graphs)

Creating Project...
Successfully created the directory static/projects/Teapot_test03 
PROGRESS: stored graph data...
PROGRESS: stored layouts...
PROGRESS: stored nodeinfo...
PROGRESS: made node position textures...
PROGRESS: made textures for node colors...
PROGRESS: stored link textures...


KeyboardInterrupt: 

# BACKGROUND INFO: 
Find the graph structure generated in this notebook based on nx.Graph input below. 

In [16]:
'''

{
----------------------------------------
THIS IS THE GENERAL GRAPH INFO SECTION
----------------------------------------
  "directed": false,
  "multigraph": false,
  "projectname": "Testgraph",
  "info": "A toy graph for testing purposes. Number of nodes: 10, Links: 43.",
  "graphlayouts": [
      "layout1-spring",
      "layout2-spring",
      "layout3-spring",
      "layout4-clusters"
  ],
  "annotationTypes": true,
  "nodes": [
   ----------------------------------------
   contains all nodes of the project
   ----------------------------------------
      {
          "id": 0,
          "name": 0,
          "annotation": 
                {
                    "annot1": [
                        "lambda",
                        "alpha",
                        "zeta",
                        "theta"
                    ],
                    "annot2": [
                        "delta",
                        "nu"
                    ],
                    "annot3": [
                        "mu",
                        "gamma"
                    ]
          }
      },....
  ],
  "links": [
   ----------------------------------------
   contains all links of the project
   ----------------------------------------
      {
          "id": 0,
          "source": 0,
          "target": 1
      },
      {
          "id": 1,
          "source": 0,
          "target": 2
      },...
       ],
  "layouts": [
   ----------------------------------------
   contains all layouts of the project
   only contains nodes and links as well as colors specific to the layout
   ----------------------------------------
       {  "layoutname" : "name of first layout",
          "nodes": [
              {
                  "nodecolor": [
                      255,
                      35,
                      0,
                      120
                  ],
                  "pos": [
                      -0.5618057865250979,
                      0.1467411221839164,
                      0.49656801102094605
                  ],
                  "id": 0
               },...
        	],
          "links": [
              {
                  "linkcolor": [
                      0,
                      255,
                      0,
                      100
                  ],
                  "source": 0,
                  "target": 1
              },...
         	],
   	  }, {
          "layoutname" : "name of second layout",
          "nodes": [
              {
                  "nodecolor": "#0000ffaa",
                  "pos": [
                      -0.35948900932978317,
                      0.6255258442839948,
                      -0.04209289102217994
                  ],
                  "cluster": "cluster group 1",
                  "id": 0
               },... 
],
          "links": [
              {
                  "linkcolor": "#0000ff",
                  "source": 0,
                  "target": 1
              },
],
  	   }, { . . .  
 	},
}

'''

'\n\n{\n----------------------------------------\nTHIS IS THE GENERAL GRAPH INFO SECTION\n----------------------------------------\n  "directed": false,\n  "multigraph": false,\n  "projectname": "Testgraph",\n  "info": "A toy graph for testing purposes. Number of nodes: 10, Links: 43.",\n  "graphlayouts": [\n      "layout1-spring",\n      "layout2-spring",\n      "layout3-spring",\n      "layout4-clusters"\n  ],\n  "annotationTypes": true,\n  "nodes": [\n   ----------------------------------------\n   contains all nodes of the project\n   ----------------------------------------\n      {\n          "id": 0,\n          "name": 0,\n          "annotation": \n                {\n                    "annot1": [\n                        "lambda",\n                        "alpha",\n                        "zeta",\n                        "theta"\n                    ],\n                    "annot2": [\n                        "delta",\n                        "nu"\n                    ],\n    

## CREATE A JSON FILE WITH THE ABOVE STRUCTURE to then create a project

In [46]:
import networkx as nx
import json 
import os

# these are the two functions one needs to create a JSON file to upload and then create the project in the backend 
import nx2json as nx2j 
import uploaderGraph as uG

In [47]:
# ----------------------------------------
# CREATE Json file
# ----------------------------------------
merged_graphs = nx2j.make_json(Graphs)
path = "temp_files/"

# save the merged graph in a json file
with open(path+Graphs[0].graph['projectname']+'.json', 'w') as fp:
    json.dump(merged_graphs, fp, indent=4)

In [ ]:
# # ----------------------------------------
# # READ Json file
# # ----------------------------------------
# filename = 'myfile.json'
# currentwd = '.../DataDiVR_Webapp/temp_files/' # modify file location here
# path = os.path.join(currentwd, filename)

# # open the json file
# with open(path, 'r') as f:
#      G_merged = json.load(f)

In [ ]:
## ----------------------------------------
# CREATE A PROJECT for the VR Platform 
# ----------------------------------------
#the actual "upload step" to create a project with the required VR platform files 

uG.upload_filesJSON(merged_graphs)